In [ ]:
import numpy as np
import math
from keras import optimizers
from SBU_utils import *
from model import *

In [ ]:
os.mkdir('SBU')
SBU_dir ='/home/SBU/'

In [ ]:
link = []
link.append('http://vision.cs.stonybrook.edu/~kiwon/Datasets/SBU_Kinect_Interactions/s01s02.zip')
link.append('http://vision.cs.stonybrook.edu/~kiwon/Datasets/SBU_Kinect_Interactions/s01s03.zip')
link.append('http://vision.cs.stonybrook.edu/~kiwon/Datasets/SBU_Kinect_Interactions/s01s07.zip')
link.append('http://vision.cs.stonybrook.edu/~kiwon/Datasets/SBU_Kinect_Interactions/s02s01.zip')
link.append('http://vision.cs.stonybrook.edu/~kiwon/Datasets/SBU_Kinect_Interactions/s02s03.zip')
link.append('http://vision.cs.stonybrook.edu/~kiwon/Datasets/SBU_Kinect_Interactions/s02s06.zip')
link.append('http://vision.cs.stonybrook.edu/~kiwon/Datasets/SBU_Kinect_Interactions/s02s07.zip')
link.append('http://vision.cs.stonybrook.edu/~kiwon/Datasets/SBU_Kinect_Interactions/s03s02.zip')
link.append('http://vision.cs.stonybrook.edu/~kiwon/Datasets/SBU_Kinect_Interactions/s03s04.zip')
link.append('http://vision.cs.stonybrook.edu/~kiwon/Datasets/SBU_Kinect_Interactions/s03s05.zip')
link.append('http://vision.cs.stonybrook.edu/~kiwon/Datasets/SBU_Kinect_Interactions/s03s06.zip')
link.append('http://vision.cs.stonybrook.edu/~kiwon/Datasets/SBU_Kinect_Interactions/s04s02.zip')
link.append('http://vision.cs.stonybrook.edu/~kiwon/Datasets/SBU_Kinect_Interactions/s04s03.zip')
link.append('http://vision.cs.stonybrook.edu/~kiwon/Datasets/SBU_Kinect_Interactions/s04s06.zip')
link.append('http://vision.cs.stonybrook.edu/~kiwon/Datasets/SBU_Kinect_Interactions/s05s02.zip')
link.append('http://vision.cs.stonybrook.edu/~kiwon/Datasets/SBU_Kinect_Interactions/s05s03.zip')
link.append('http://vision.cs.stonybrook.edu/~kiwon/Datasets/SBU_Kinect_Interactions/s07s02.zip')
link.append('http://vision.cs.stonybrook.edu/~kiwon/Datasets/SBU_Kinect_Interactions/s06s03.zip')
link.append('http://vision.cs.stonybrook.edu/~kiwon/Datasets/SBU_Kinect_Interactions/s06s04.zip')
link.append('http://vision.cs.stonybrook.edu/~kiwon/Datasets/SBU_Kinect_Interactions/s07s01.zip')
link.append('http://vision.cs.stonybrook.edu/~kiwon/Datasets/SBU_Kinect_Interactions/s07s03.zip')

import wget
for i in link:
    temp_path = SBU_dir + i.split('/')[-1]
    url = i
    print(url)
    print(temp_path)
    wget.download(url=url, out=temp_path)

In [ ]:
!unzip '/content/SBU/s07s01.zip'
!unzip '/content/SBU/s07s03.zip'
!unzip '/content/SBU/s06s04.zip'
!unzip '/content/SBU/s06s03.zip'
!unzip '/content/SBU/s06s02.zip'
!unzip '/content/SBU/s05s03.zip'
!unzip '/content/SBU/s05s02.zip'
!unzip '/content/SBU/s04s02.zip'
!unzip '/content/SBU/s04s03.zip'
!unzip '/content/SBU/s04s06.zip'
!unzip '/content/SBU/s03s02.zip'
!unzip '/content/SBU/s03s04.zip'
!unzip '/content/SBU/s03s05.zip'
!unzip '/content/SBU/s03s06.zip'
!unzip '/content/SBU/s02s01.zip'
!unzip '/content/SBU/s02s03.zip'
!unzip '/content/SBU/s02s06.zip'
!unzip '/content/SBU/s02s07.zip'
!unzip '/content/SBU/s01s03.zip'
!unzip '/content/SBU/s01s07.zip'
!unzip '/content/SBU/s01s02.zip'

In [ ]:
dataset = SBU_dataset('/home')
train, test = dataset.get_data(3)

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
lr=0.001
adam = optimizers.Adam(lr)
model = multi_obj()
model.compile(adam, loss='mean_squared_error')
model.summary()

In [ ]:
epochs = 400
import random
for e in range(epochs):
    X_0 = []
    X_1 = []
    X_2 = []
    X_3 = []
    Y = []

    for i in range(1,9):                 # loop 8 classes
        for j in range(len(train[i])):   # loop all samples within the same class
            
            #First person pose
            p_0 = np.copy(train[i][j].T[:,:45])
            p_0 = p_0.reshape([-1,15,3])
            t_0 = p_0.shape[0]           # the number of all frames
            if t_0>16:                   # sample the range from crop size of [16,t_0]
                ratio = np.random.uniform(1,t_0/16)
                l = int(16*ratio)
                start = random.sample(range(t_0-l),1)[0]
                end = start+l
                p_0 = p_0[start:end,:,:]
                p_0 = zoom(p_0)
            elif t_0<16:
                p_0 = zoom(p_0)
            
            #Calculate the temporal difference
            p_0_diff = p_0[1:,:,:]-p_0[:-1,:,:]
            p_0_diff = np.concatenate((p_0_diff,np.expand_dims(p_0_diff[-1,:,:],axis=0)))
            #p_1_diff = p_1[1:,:,:]-p_1[:-1,:,:]
            #p_1_diff = np.concatenate((p_1_diff,np.expand_dims(p_1_diff[-1,:,:],axis=0)))

            X_0.append(p_0)
            X_1.append(p_0_diff)
            #X_2.append(p_1)
            #X_3.append(p_1_diff)

            label = np.zeros(8)
            label[i-1] = 1
            Y.append(label)

    X_0 = np.stack(X_0)
    X_1 = np.stack(X_1)
    #X_2 = np.stack(X_2)
    #X_3 = np.stack(X_3)
    Y = np.stack(Y)    

    history = model.fit([X_0,X_1],Y,batch_size=32,epochs=1,verbose=True,shuffle=True)
    
    if not (e+1)%50:
        lr *= 0.8
        adam = optimizers.Adam(lr)
        model.compile(adam, loss='mean_squared_error')

In [ ]:
X_TEST_0 = []
X_TEST_1 = []
X_TEST_2 = []
X_TEST_3 = []
Y_TEST = []

for i in range(1,9):
    for j in range(len(test[i])):
        p_0 = np.copy(test[i][j].T[:,:45])
        p_0 = p_0.reshape([-1,15,3])
        t_0 = p_0.shape[0]
        # if the number of frame is more than 20, crop by scale 0.9, then rescale by interploration again
        if t_0>=20: 
            p_0 = p_0[int(t_0*0.05):int(t_0*0.95),:,:]
            p_0 = zoom(p_0)
        elif t_0<20:
            p_0 = zoom(p_0)
        p_0_diff = p_0[1:,:,:]-p_0[:-1,:,:]
        p_0_diff = np.concatenate((p_0_diff,np.expand_dims(p_0_diff[-1,:,:],axis=0)))
        
        X_TEST_0.append(p_0)
        X_TEST_1.append(p_0_diff)
        
        label = np.zeros(8)
        label[i-1] = 1
        Y_TEST.append(label)

X_TEST_0 = np.stack(X_TEST_0)
X_TEST_1 = np.stack(X_TEST_1)
Y_TEST = np.stack(Y_TEST)

Y_pred = model.predict([X_TEST_0,X_TEST_1])

print('Predict labels:',np.argmax(Y_pred,axis=1))
print('Ground truth labels:',np.argmax(Y_TEST,axis=1))